In [ ]:
# default_exp core

# pySplitting

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
print(1+4)

5


In [ ]:
from sympy import factorial as fact
import array as arr
#x = symbols('x')
#a = Integral(cos(x)*exp(x),x)
#Eq(a,a.doit())

In [ ]:
#solve(x**2-1.5,x)

In [ ]:
%nbdev_export

from sympy import factorial as fact

# Mathematica counts from 1, python from zero!!!!!!
def findpos(lam):
    retval = -1
    i = 0
    while (i < len(lam)-1) and (retval == -1):
        if lam[i] == lam[i+1]:
            i = i + 1
        else:
            retval = i
    if retval == -1:
        retval = len(lam)-1
    return retval

In [ ]:
l=arr.array('i',[5,3,3])
print("res=", findpos(l))

res= 0


In [ ]:
len(l)

3

In [ ]:
%nbdev_export
def isstrictlyascending(lam):
    """This determines whether the index string is a strictly ascending sequence
    
    Parameters
    ----------
    lam : an indexable object of things that are comparable. Usually an array of integers
    
    Returns
    -------
    bool : true if the sequence is strictly ascending, else false.
    """
    retval = True
    i = 0
    while (i < len(lam)-1) and retval == True:
        if lam[i] < lam[i+1]:
            i = i+1
        else:
            retval = False
    return retval

In [ ]:
l=arr.array('i',[2,3,4])
print("res=", isstrictlyascending(l))

res= True


In [ ]:
%nbdev_export
def alpha(lam):
    retval = 1
    if (len(lam) > 1) and not isstrictlyascending(lam):
        pos = findpos(lam)
        retval = 1/fact(pos+1)*alpha(lam[pos+1:])
    return retval

In [ ]:
l=arr.array('i',[1,2,3,3])
print("res= ",alpha(l))

res=  1/2


In [ ]:
t = arr.array('i',[2,2])
print(t[1+1:])

array('i')


In [ ]:
alpha([1,1,2])

1/2

In [ ]:
fact(34)

295232799039604140847618609643520000000